In [1]:
import requests
import time, os
import random
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import pandas as pd
from IPython.core.display import display, HTML

In [2]:
''' Look at 2019, look at 2002
add from both
look at median year, add
then start checking between 2019 and median year?

But for actually grabbing the data... this is going to be a bit confusing
Maybe I need to just click back for 10 years and scrap this craping idea below.
'''

' Look at 2019, look at 2002\nadd from both\nlook at median year, add\nthen start checking between 2019 and median year?'

Average QB career is 3.3 years. Perhaps capture QBs who have been active in the last 5 years.

In [123]:
def qb_soup_scraper():
    '''
    Create a list of soup files to search through. Adjust years_list variable to adjust years to scrape.
    return:
        list of soup files that contain QB names
    '''
    base_url = 'https://www.pro-football-reference.com/years/'
    html = '/passing.htm'
    years_list = list(map(str,[x for x in range(2010,2019)]))
    ua = UserAgent()
    user_agent = {'User-agent' : ua.random}
    
    soup_list=[]
    
    for year in years_list:
        response = requests.get(base_url + year + html, headers = user_agent)
        # I would insert a try and raise test here to check response but nervouse about scraping page. Try later.
        soup = BeautifulSoup(response.text, 'lxml')
        soup_list.append(soup)
        time.sleep(.5+2*random.random())
    return soup_list

In [124]:
soup_list = qb_soup_scraper()

In [208]:
def raw_qb_list(soup):
    qb_list = []
    links = soup.find_all('tbody')
    for qbpos in links:
        qb_names = qbpos.find_all('a')
        for names in qb_names:
            str_test = names.text.split()
            if len(str_test) > 1:
                qb_list.append(str_test)
    qb_list = [' '.join(name) for name in qb_list]
    return qb_list

In [193]:
def raw_pos_list(soup):
    position_list=[]
    pos_soup = soup.find_all('td', {'data-stat':'pos'})
    for x in pos_soup:
        position_list.append(x.text.lower())
    return position_list

In [221]:
def raw_scrape(soup_list):
    qb_filter_dict = {}
    qb_master_list = []
    qb_temp_list = []
    pos_list = []
    
    for soup in soup_list:
        qb_temp_list = raw_qb_list(soup)
        pos_list = raw_pos_list(soup)
        # Create df to filter either obscure or non-QBs
        filter_df = pd.DataFrame()
        filter_df['name'] = qb_temp_list
        filter_df['pos'] = pos_list
        filter_df = filter_df[filter_df['pos'] == 'qb']
        qb_temp_list = list(filter_df['name'])
        for qb in qb_temp_list:
            if qb not in qb_master_list:
                qb_master_list.append(qb)
    
    return qb_master_list

In [222]:
qb_master_list = raw_scrape(soup_list)

102
102
96
96
85
85
85
85
100
100
86
86
96
96
94
94
106
106
